In [52]:
import numpy as np
import re
import random

In [6]:
def getNodes(graph):
    nodes = []
    for n in graph.keys():
        nodes.append(n)
    for n2 in graph.values():
        if type(n2) == "string":
            nodes.append(n2)
        else:
            nodes += n2
    return np.unique(nodes).tolist()
    

def getEdges(graph, isDirected):
    edges = []
    for item in graph.items():
        for destination in item[1]:
            if (not isDirected):
                string1 = item[0] if item[0] < destination else destination
                string2 = destination if item[0] < destination else item[0]
            else:
                string1 = item[0]
                string2 = destination
            edges.append(string1 + string2)
    return np.unique(edges).tolist()

In [7]:
graph0 = {
    "a": ["b"]
}
#A---B

graph1 = {
    "a": ["b"],
    "b": ["c"]
}
#A----B----C   D=v0

graph2 = {
    "a": ["b", "c", "d"],
    "b": ["a"],
    "c": ["a"],
    "d": ["a"]
}

#       C          D
#        \        /
#         \      /
#          \    /
#           \  /
# B----------A

graph3 = {
    "a": "b",
    "b": "c",
    "c": "d"
}
# A--B--C--D

graph4 = {
    "a": "b",
    "b": "c",
    "c": "d",
    "d": "e"
}
# A--B--C--D--E

graph5 = {
    "a": ["b", "c", "d", "e"]
}
#       C          D
#        \        /
#         \      /
#          \    /
#           \  /
# B----------A-------------E

In [106]:
class Player:
    def __init__(self, types, shorthand, roundNum, isCover=False):
        self.types = types
        self.shorthand = shorthand
        self.roundNum = roundNum
        self.isCover = isCover
    def self_introduce(self):
        return f'Player {self.shorthand}'

def buildPlayers(nodes, edges, covers):
    # objectives
    objectivelayer = [Player('objective', 'o', 1)]
    # nodes
    n = len(nodes)
    vertexplayers = [Player('vertex', 'v-0', 1)]
    for i in nodes:
        vertexplayers.append(Player('vertex', f'v-{i}', 1, i in covers))
    # edges
    m = len(edges)
    edgeplayers = []
    for j in edges:
        edgeplayers.append(Player('edge', f'e-{j}', 1))
    # filler    
    k = len(covers)
    roundLimit1 = int(np.ceil(np.log(n-k)))
    fillerplayers = []
    for r in range(1, roundLimit1+1):
        for i in range(1,k+1):
            #for j in vertexplayers:
            fillerplayers.append(Player('filler', f'f^{r}_v#{i}', r))
    roundLimit2 = int(np.ceil(np.log(n-k)) + np.ceil(np.log(m)))
    for r in range(roundLimit1+1, roundLimit2+1):
        for i in range(1,k+1):
            #for j in edgeplayers:
            fillerplayers.append(Player('filler', f'f^{r}_e#{i}', r))
    # holder
    holderplayers = []
    for ei in edgeplayers:
        for l in range(0, 2**roundLimit1 - 1):
            holderplayers.append(Player('holder', f'h^{l}_{ei.shorthand}', 1))
            
    for fi in fillerplayers:
        fillerplayerR = fi.roundNum
        for l in range(0, 2**(fillerplayerR -1) -1):
            holderplayers.append(Player('holder', f'h^{l}_{fi.shorthand}', 1))
    kfiller = 2**(roundLimit2 + int(np.ceil(np.log(k+1)))+1) - 1
    for l in range(0, kfiller):
        holderplayers.append(Player('holder', f'h^{l}_o', 1))
    return {
        'objectivePlayer': objectivelayer,
        'vertexPlayers': vertexplayers, 
        'edgePlayers': edgeplayers,
        'fillerPlayers': fillerplayers,
        'holderPlayers': holderplayers
    }

In [107]:
chosenGraph = graph1
nodes = getNodes(chosenGraph)
edges = getEdges(chosenGraph, False)
covers = ["b"]
players = buildPlayers(nodes, edges, covers)

#rounds of number of players
def getFullPlayers(playerNums):
    n = 0
    stopFlag = False
    while (not stopFlag):
        if playerNums > 2**n:
            n += 1
        else:
            stopFlag = True
    return 2**n

def fillPlayers(playersList):
    playerNums = np.sum([
        len(playersList['objectivePlayer']),
        len(playersList['vertexPlayers']), 
        len(playersList['edgePlayers']),
        len(playersList['fillerPlayers']),
        len(playersList['holderPlayers'])
    ])
    if getFullPlayers(playerNums) > playerNums:
        for i in range(0, getFullPlayers(playerNums) - playerNums):
            players['holderPlayers'].append(Player('holder', f'h^{i}_o_ammend', 1))
    return playersList

In [108]:
players =fillPlayers(players)

In [109]:
for ptype in players.values():
    for p in ptype:
        print(p.self_introduce())

Player o
Player v-0
Player v-a
Player v-b
Player v-c
Player e-ab
Player e-bc
Player f^1_v#1
Player f^2_e#1
Player h^0_e-ab
Player h^0_e-bc
Player h^0_f^2_e#1
Player h^0_o
Player h^1_o
Player h^2_o
Player h^3_o
Player h^4_o
Player h^5_o
Player h^6_o
Player h^7_o
Player h^8_o
Player h^9_o
Player h^10_o
Player h^11_o
Player h^12_o
Player h^13_o
Player h^14_o
Player h^0_o_ammend
Player h^1_o_ammend
Player h^2_o_ammend
Player h^3_o_ammend
Player h^4_o_ammend


In [110]:
def pair(p1, p2, r1, r2, matches):
    matches.append((p1, p2))
    if p1 in r1:
        r1.remove(p1)
    if p2 in r2:
        r2.remove(p2)
    
def matchingPhase1(playerList, r):
    seeds = []
    objective = playerList['objectivePlayer'][0]
    holderString = playerList['holderPlayers']
    remainingHolders = holderString.copy()
    
    vertexString = playerList['vertexPlayers']
    remainingVertices = vertexString.copy()
    
    edgeString = playerList['edgePlayers']
    remainingEdges = edgeString.copy()
    
    fillerString = playerList['fillerPlayers']
    remainingFillers = fillerString.copy()
    
    matchedPlayers = []
    for vertexPlayer in vertexString:
        if vertexPlayer.isCover:
            for fillerPlayer in fillerString:
                if "v" in fillerPlayer.shorthand and fillerPlayer.roundNum == r and fillerPlayer not in matchedPlayers and vertexPlayer not in matchedPlayers:
                    pair(fillerPlayer, vertexPlayer, remainingFillers, remainingVertices, seeds)
                    matchedPlayers.append(fillerPlayer)
                    matchedPlayers.append(vertexPlayer)
        else:
            for vertexPlayer2 in remainingVertices:
                matchCountvertexPlayer2 = 0
                if vertexPlayer.shorthand != vertexPlayer2.shorthand and not vertexPlayer2.isCover and vertexPlayer2 not in matchedPlayers and vertexPlayer not in matchedPlayers:
                    pair(vertexPlayer, vertexPlayer2, remainingVertices, remainingVertices, seeds)
                    matchedPlayers.append(vertexPlayer)
                    matchedPlayers.append(vertexPlayer2)
    fillerString = remainingFillers.copy()
    vertexString = remainingVertices.copy()
    
    for edgePlayer in edgeString:
        for holdPlayer in holderString:
            if edgePlayer.shorthand in holdPlayer.shorthand and edgePlayer not in matchedPlayers and holdPlayer not in matchedPlayers:
                pair(edgePlayer, holdPlayer, remainingEdges, remainingHolders, seeds)
                matchedPlayers.append(edgePlayer)
                matchedPlayers.append(holdPlayer)
    holderString = remainingHolders.copy()
    edgeString = remainingEdges.copy()
    
    for fillerPlayer in fillerString:
        matchCount = 0
        for holdPlayer in holderString:
            if fillerPlayer.shorthand in holdPlayer.shorthand and matchCount == 0 and fillerPlayer not in matchedPlayers and holdPlayer not in matchedPlayers:
                matchCount += 1
                pair(fillerPlayer, holdPlayer, remainingFillers, remainingHolders, seeds)
                matchedPlayers.append(fillerPlayer)
                matchedPlayers.append(holdPlayer)
    holderString = remainingHolders.copy()
    fillerString = remainingFillers.copy()
    
    seeds.append((objective, holderString[-1]))
    
    #straggers:
    remainings = remainingFillers + remainingHolders + remainingEdges + remainingVertices;
    while len(remainings) >= 2:
        selectPairs = random.sample(remainings, 2)
        pair(selectPairs[0], selectPairs[1], remainings, remainings, seeds)
    return seeds

In [111]:
round1 = matchingPhase1(players, 1)
for i in round1:
    print(f'{i[0].self_introduce()} vs. {i[1].self_introduce()}')

Player v-0 vs. Player v-a
Player f^1_v#1 vs. Player v-b
Player e-ab vs. Player h^0_e-ab
Player e-bc vs. Player h^0_e-bc
Player f^2_e#1 vs. Player h^0_f^2_e#1
Player o vs. Player h^4_o_ammend
Player h^1_o_ammend vs. Player h^10_o
Player h^3_o vs. Player v-c
Player h^14_o vs. Player h^5_o
Player h^6_o vs. Player h^4_o_ammend
Player h^9_o vs. Player h^7_o
Player h^3_o_ammend vs. Player h^4_o
Player h^2_o vs. Player h^0_o
Player h^13_o vs. Player h^0_o_ammend
Player h^11_o vs. Player h^12_o
Player h^8_o vs. Player h^2_o_ammend


In [112]:
def decider(player1, player2):
    type1 = player1.types
    type2 = player2.types
    if type1 == 'objective':
        if type2 == 'vertex':
            return player1
        elif type2 == 'edge':
            return player2
        elif type2 == 'filler':
            return player2
        elif type2 == 'holder':
            if 'o' in player2.shorthand:
                return player1
            else:
                return player2
            
    elif type1 == 'vertex':
        if type2 == 'vertex':
            if player1.shorthand < player2.shorthand:
                return player1
            else:
                return player2
        elif type2 == 'edge':
            vertice = re.search(r"v-(\w+)", player1.shorthand).group(1)
            edge = re.search(r"e-(\w+)", player2.shorthand).group(1)
            if vertice in edge:
                return player1
            else:
                return player2
        elif type2 == 'filler':
            return player1
        elif type2 == 'holder':
            return player2
        
    elif type1 == 'edge':
        if type2 == 'vertex':
            vertice = re.search(r"v-(\w+)", player1.shorthand).group(1)
            edge = re.search(r"e-(\w+)", player2.shorthand).group(1)
            if vertice in edge:
                return player2
            else:
                return player1
        if type2 == 'edge':
            return random.choice([player1, player2])
        if type2 == 'filler':
            return random.choice([player1, player2])
        if type2 == 'holder':
            if 'e-' in player2.shorthand:
                if player1.shorthand in player2.shorthand:
                    return player1
                else:
                    return random.choice([player1, player2])
            else:
                return player2
    
    elif type1 == 'filler':
        if type2 == 'vertex':
            return player2
        elif type2 == 'edge':
            return random.choice([player1, player2])
        elif type2 == 'filler':
            return random.choice([player1, player2])
        elif type2 == 'holder':
            if 'e-' in player2.shorthand:
                return random.choice([player1, player2])
            elif  'o' in player2.shorthand or  player1.shorthand in player2.shorthand:
                return player1
            else:
                return random.choice([player1, player2])
            
    elif type1 == 'holder':
        if 'e-' in player1.shorthand:
            if type2 == 'vertex':
                return player1
            elif type2 == 'edge':
                if player2.shorthand in player1.shorthand:
                    return player2
                else:
                    return random.choice([player1, player2])
            elif type2 == 'filler':
                return random.choice([player1, player2])
            elif type2 == 'holder':
                if 'o' in player2.shorthand:
                    return player1
                else:
                    return random.choice([player1, player2])
        elif 'f^' in player1.shorthand:
            if type2 == 'vertex':
                return player1
            elif type2 == 'edge':
                return player2
            elif type2 == 'filler':
                if player2.shorthand in player1.shorthand:
                    return player2
                else:
                    return random.choice([player1, player2])
            elif type2 == 'holder':
                if 'o' in player2.shorthand:
                    return player1
                else:
                    return random.choice([player1, player2])
        elif 'o' in player1.shorthand:
            if type2 == 'vertex':
                return player1
            elif type2 == 'edge':
                return player2
            elif type2 == 'filler':
                return player2
            elif type2 == 'holder':
                if 'o' in player2.shorthand:
                    return random.choice([player1, player2])
                else:
                    return player2

In [115]:
result = []
for match in round1:
    result.append(decider(match[0], match[1]))

for p in result:
    print(p.self_introduce())

Player v-0
Player v-b
Player e-ab
Player e-bc
Player f^2_e#1
Player o
Player h^1_o_ammend
Player h^3_o
Player h^14_o
Player h^6_o
Player h^9_o
Player h^3_o_ammend
Player h^0_o
Player h^13_o
Player h^11_o
Player h^8_o


In [104]:
result

In [ ]:
# idea comes from this very very old paper
# https://www.barton.edu/pdf/faculty-publications/bengtson-winning-probabilities-publication.pdf

import numpy as np

p = [[1, 0.434, 0.337, 0.298, 0.262, 0.277, 0.312, 0.485],
[0.566, 1, 0.571, 0.503, 0.42, 0.456, 0.516, 0.546],
[0.663, 0.429, 1, 0.432, 0.356, 0.389, 0.463, 0.424],
[0.702, 0.497, 0.568, 1, 0.416, 0.454, 0.514, 0.492],
[0.738, 0.58, 0.644, 0.584, 1, 0.592, 0.596, 0.576],
[0.723, 0.544, 0.611, 0.546, 0.408, 1, 0.56, 0.538],
[0.688, 0.484, 0.537, 0.486, 0.404, 0.44, 1, 0.478],
[0.515, 0.454, 0.576, 0.508, 0.424, 0.462, 0.522, 1]]



# bracket kingstone is making
tournament = [[(0,1),(2,3),(4,5),(6,7)],#round 1 
             [(0,2),(4,6)],#round 2
             [(0,4)]]#round3

#gives probability of winnig i agains j (philip func)
def P(i,j):
    return p[i][j]
 
#just a function can go through braket and find the opponent
def opponent(player_i,round):
    for (i,j) in tournament[round-1]:
        if i == player_i:
            return j
        elif j == player_i:
            return i
    return None

#go to the refrence for this one
def Win_rate(player_i,round):
    if round == 0 : return 1
    opp_player = opponent(player_i,round)
    if opp_player == None: return 0
    return Win_rate(player_i,round-1) * P(player_i,opp_player) * Win_rate(opp_player,round-1)

Win_rate(2,3)

0